<img src="img/pandas_logo.png" align="right" width="40%">

DataFrame
==========

The `dask.dataframe` module implements a blocked parallel `DataFrame` object that mimics a subset of the Pandas `DataFrame`. One dask `DataFrame` is comprised of several in-memory pandas `DataFrames` separated along the index. One operation on a dask `DataFrame` triggers many pandas operations on the constituent pandas `DataFrame`s in a way that is mindful of potential parallelism and memory constraints.

**Related Documentation**

*  [Dask DataFrame documentation](http://dask.pydata.org/en/latest/dataframe.html)
*  [Pandas documentation](http://pandas.pydata.org/)

**Main Take-aways**

1.  Dask.dataframe should be familiar to Pandas users
2.  The index grows to include partitions, which are important for efficient queries

### Setup

We create artifical data.

In [ ]:
from prep import accounts_csvs
accounts_csvs(3, 5000000, 500)

In [ ]:
import os
filename = os.path.join('data', 'accounts.*.csv')

## Load Data from CSVs and inspect the dask graph

### `dask.dataframe.read_csv`

This works just like `pandas.read_csv`, except on multiple csv files at once.

In [ ]:
filename

In [ ]:
import dask.dataframe as dd
df = dd.read_csv(filename)

In [ ]:
df

However unlike Pandas, operations on dask.dataframes don't trigger immediate computation, instead they add key-value pairs to an underlying dask graph.

Some operations trigger direct computation, like `len` and `head`:

In [ ]:
%time len(df)

In [ ]:
df.head()

In [ ]:
pd.options.display.memory_usage = 'deep'

In [ ]:
df.info(memory_usage=True)

### How does this compare to Pandas?

#### Features and Size

Pandas is more mature and fully featured than `dask.dataframe`.  If your data fits in memory then you should use Pandas.  The `dask.dataframe` module gives you a limited `pandas` experience when you operate on datasets that don't fit comfortably in memory.

During this tutorial we provide a small dataset consisting of a few CSV files.  This dataset is 45MB on disk that expands to about 400MB in memory (the difference is caused by using `object` dtype for strings).  This dataset is small enough that you would normally use Pandas.

We've chosen this size so that exercises finish quickly.  Dask.dataframe only really becomes meaningful for problems significantly larger than this, when Pandas breaks with the dreaded 

    MemoryError:  ...
    
#### Speed

Dask.dataframe operations use `pandas` operations internally.  Generally they run at about the same speed except in the following two cases:

1.  Dask introduces a bit of overhead, around 1ms per task.  This is usually negligible.
2.  When Pandas releases the GIL (coming to `groupby` in the next version) `dask.dataframe` can call several pandas operations in parallel increasing speed somewhat proportional to the number of cores.

Exercise: Recall and use Pandas API
----------------------------------------

If you are already familiar with the Pandas API then you should have a firm grasp on how to use `dask.dataframe`.  There are a couple of small changes.

As noted above, computations on dask `DataFrame` objects don't perform work, instead they build up a dask graph.  We can evaluate this dask graph at any time using the `.compute()` method.

In [ ]:
result = df.amount.mean()  # create lazily evaluated result
result

In [ ]:
result.compute()           # perform actual computation

<div class="alert alert-success">
    <b>EXERCISE</b>: Using groupby(), calculate the average age for each sex.

Try the following exercises

<ol>
<li> Use the `head()` method to get the first ten rows. </li> 
<li> Use the `drop_duplicates()` method to find all of the distinct names</li>
<li> Use selections `df[...]` to find how many positive and negative amounts there are</li>
<li> Use groupby `df.groupby(df.A).B.func()` to get the average amount per user ID</li>
<li> Sort the result to (4) by amount, find the names of the top 10 </li>
</ol>

<p> This section should be easy if you are familiar with Pandas.  If you aren't then that's ok too.  You may find the [pandas documenation](http://pandas.pydata.org/) a useful read in the future.  Don't worry, future sections in this tutorial will not depend on this knowledge. </p>

</div>


In [ ]:
# 1. Use the `head()` method to get the first ten rows
#    Note, head computes by default, this is the only operation that doesn't need an explicit call to .compute()


In [ ]:
# 2. Use the `drop_duplicates()` method to find all of the distinct names


In [ ]:
# 3. Use selections `df[...]` to find how many positive and negative amounts there are


In [ ]:
# 3. Use selections `df[...]` to find how many positive and negative amounts there are


In [ ]:
# 4. Use groupby `df.groupby(df.A).B.func()` to get the average amount per user ID 


In [ ]:
# 5. Combine your answers to 3 and 4 to compute the average withdrawal (negative amount) per name


<!--    <img src="img/dask-dataframe.svg" align="right" width="40%"> -->

Divisions and the Index
---------------------------

The Pandas index associates a value to each record/row of your data.  Operations that align with the index, like `loc` can be a bit faster as a result.

In `dask.dataframe` this index becomes even more important.  Recall that one dask `DataFrame` consists of several Pandas `DataFrame`s.  These dataframes are separated along the index by value.  For example, when working with time series we may partition our large dataset by month.

Recall that these many partitions of our data may not all live in memory at the same time, instead they might live on disk; we simply have tasks that can materialize these pandas `DataFrames` on demand.

Partitioning your data can greatly improve efficiency.  Operations like `loc`, `groupby`, and `merge/join` along the index are *much more efficient* than operations along other columns.  You can see how your dataset is partitioned with the `.divisions` attribute.  Note that data that comes out of simple data sources like CSV files aren't intelligently indexed by default.  In these cases the values for `.divisions` will be `None.`

In [ ]:
df = dd.read_csv(filename)
df.divisions

In [ ]:
df

However if we set the index to some new column then dask will divide our data roughly evenly along that column and create new divisions for us.  Warning, `set_index` triggers immediate computation.

In [ ]:
df2 = df.set_index('names')
df2.divisions

In [ ]:
df2.head()

In [ ]:
len(df2)

We see here the minimum and maximum values ("Alice" and "Zelda") as well as several intermediate values that separate our data well. 

In [ ]:
df2.npartitions

In [ ]:
df2.head()

Operations like `loc` only need to load the relevant partitions

In [ ]:
df2.loc['Edith']

In [ ]:
df2.loc['Edith'].compute()

## DataFrame Storage


Efficient storage can dramatically improve performance, particularly when operating repeatedly from disk.

Decompressing text and parsing CSV files is expensive.  One of the most effective strategies with medium data is to use a binary storage format like HDF5.  Often the performance gains from doing this is sufficient so that you can switch back to using Pandas again instead of using `dask.dataframe`.

In this section we'll learn how to efficiently arrange and store your datasets in on-disk binary formats.  We'll use the following:

1.  [Pandas `HDFStore`](http://pandas.pydata.org/pandas-docs/stable/io.html#io-hdf5) format on top of `HDF5`
2.  Categoricals for storing text data numerically

**Main Take-aways**

1.  Storage formats affect performance by an order of magnitude
2.  Text data will keep even a fast format like HDF5 slow
3.  A combination of binary formats, column storage, and partitioned data turns one second wait times into 80ms wait times.

### Write to HDF5

Pandas contains a specialized HDF5 format, `HDFStore`.  The ``dd.DataFrame.to_hdf`` method works exactly like the ``pd.DataFrame.to_hdf`` method.

In [ ]:
target = os.path.join('data', 'accounts.h5')
target

In [ ]:
%time df.to_hdf(target, '/data')

In [ ]:
df2 = dd.read_hdf(target, '/data')
df2.head()

### Compare CSV to HDF5 speeds

We do a simple computation that requires reading a column of our dataset and compare performance between CSV files and our newly created HDF5 file.  Which do you expect to be faster?

In [ ]:
%time df.amount.sum().compute()

In [ ]:
%time df2.amount.sum().compute()

Sadly they are about the same.  

The culprit here is `names` column, which is of `object` dtype and thus hard to store efficiently.  There are two problems here:

1.  How do we store text data like `names` efficiently on disk?
2.  Why did we have to read the `names` column when all we wanted was `amount`

### 1.  Store text efficiently with categoricals

We can use Pandas categoricals to replace our object dtypes with a numerical representation.  This takes a bit more time up front, but results in better performance.

More on categoricals at the [pandas docs](http://pandas.pydata.org/pandas-docs/stable/categorical.html) and [this blogpost](http://matthewrocklin.com/blog/work/2015/06/18/Categoricals).

In [ ]:
# Categorize data, then store in HDFStore
%time df.categorize(columns=['names']).to_hdf(target, '/data2')

In [ ]:
# It looks the same
df2 = dd.read_hdf(target, '/data2')
df2.head()

In [ ]:
# But loads more quickly
%time df2.amount.sum().compute()

This is significantly faster.  This tells us that it's not only the file type that we use but also how we represent our variables that influences storage performance.

However this can still be better.  We had to read all of the columns (`names` and `amount`) in order to compute the sum of one (`amount`). For this, we could use other columnar on-disk stores like parquet. 

Limitations
-------------

### What doesn't work?

Dask.dataframe only covers a small but well-used portion of the Pandas API.
This limitation is for two reasons:

1.  The Pandas API is *huge*
2.  Some operations are genuinely hard to do in parallel (e.g. sort)

Additionally, some important operations like ``set_index`` work, but are slower
than in Pandas because they may write out to disk.

Finally, `dask.dataframe` is quite new and non-trivial bugs are frequently reported (and quickly fixed).

### What definitely works?

* Trivially parallelizable operations (fast):
    *  Elementwise operations:  ``df.x + df.y``
    *  Row-wise selections:  ``df[df.x > 0]``
    *  Loc:  ``df.loc[4.0:10.5]``
    *  Common aggregations:  ``df.x.max()``
    *  Is in:  ``df[df.x.isin([1, 2, 3])]``
    *  Datetime/string accessors:  ``df.timestamp.month``
* Cleverly parallelizable operations (also fast):
    *  groupby-aggregate (with common aggregations): ``df.groupby(df.x).y.max()``
    *  value_counts:  ``df.x.value_counts``
    *  Drop duplicates:  ``df.x.drop_duplicates()``
    *  Join on index:  ``dd.merge(df1, df2, left_index=True, right_index=True)``
* Operations requiring a shuffle (slow-ish, unless on index)
    *  Set index:  ``df.set_index(df.x)``
    *  groupby-apply (with anything):  ``df.groupby(df.x).apply(myfunc)``
    *  Join not on the index:  ``pd.merge(df1, df2, on='name')``
* Ingest operations
    *  CSVs: ``dd.read_csv``
    *  Pandas: ``dd.from_pandas``
    *  Anything supporting numpy slicing: ``dd.from_array``
    *  Dask.bag: ``mybag.to_dataframe(columns=[...])``